In [2]:
from transformers import Wav2Vec2ForCTC
from transformers import Wav2Vec2Processor
import torch
model_id = 'hongseongpil/Wav2Vec2.0_zeroth_Ko'
model = Wav2Vec2ForCTC.from_pretrained(model_id,output_attentions=True)
model.eval()
processor = Wav2Vec2Processor.from_pretrained(model_id)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
import torch
from pydub.effects import normalize
from pydub import AudioSegment 
from pydub.silence import detect_nonsilent
import numpy as np
def getAudioInput(source_path , silentremove = True):
    try:
        audio = AudioSegment.from_file(source_path).set_frame_rate(16000).set_sample_width(2).set_channels(1)
        print(audio.duration_seconds)
        newaudio = AudioSegment.empty()
        if(silentremove):
            result = detect_nonsilent(audio,min_silence_len=200,silence_thresh=-60)
            newaudio = AudioSegment.empty()
            for index in result:
                newaudio += audio[index[0]:index[1]]
            newaudio = normalize(newaudio)
        else:
            newaudio = normalize(audio)
        return newaudio
    except Exception as e:
        print("오류 발생:", e)
def PrintAudioInfo(audio):
    channels = audio.channels
    sample_rate = audio.frame_rate
    print("Channels:", channels)
    print("Sample rate:", sample_rate)
    print("Duration: ", audio.duration_seconds)
    print("Bit depth:", audio.sample_width, "bits") 
    print("len samples:", len(np.array(audio.get_array_of_samples())))
    display(audio)

In [4]:
def GetLogits(audio):
    input = processor(np.array(audio.get_array_of_samples(),dtype=np.float32), sampling_rate=16000, return_tensors="pt").input_values[0]
    with torch.no_grad():
        input_values = torch.tensor(input).unsqueeze(0)
        return model(input_values).logits
    
def DisplayPred(logits , k):
    sorted_tensor, indices = torch.sort(logits, dim=2, descending=True)
    outputs = list()
    for i,v in enumerate(indices[0]):
        outputs.append((processor.tokenizer.convert_ids_to_tokens(v[:k].tolist()),v[:k].tolist()))
    for i,v in enumerate(outputs):
        print(sorted_tensor[0][i][:k],v)

def DisplayResult(logits, audio , showaudio = False):
    predlogits = torch.argmax(logits, dim=-1)[0]
    outputs = processor.decode(predlogits,output_char_offsets=True)
    time_offset = model.config.inputs_to_logits_ratio / processor.feature_extractor.sampling_rate
    PrintAudioInfo(audio)
    print( [w['char'] for w in outputs['char_offsets']])
    for w in outputs['char_offsets']:
        start_offset = round (w[ "start_offset" ] * time_offset, 2 )
        end_offset = round (w[ "end_offset" ] * time_offset, 2 )
        print('word : ',w['char'],'start_offset :',start_offset,'end_offset :',end_offset)
        if(showaudio):
            display(audio[start_offset*1000:end_offset*1000])
    


In [5]:
from Tokenize_Kor import decompose_tokens
audio = getAudioInput("wav" , False)
logits = GetLogits(audio)
predlogits = torch.argmax(logits, dim=-1)[0]
outputs = processor.decode(predlogits,output_char_offsets=True)
origintext = "어제 친구랑 싸워서 기분이 안좋아.."
origintext = "".join(decompose_tokens(origintext)).replace(" ","")
predtext = outputs['text'].replace(" ","")

오류 발생: [Errno 2] No such file or directory: 'weather_00207003.wav'


AttributeError: 'NoneType' object has no attribute 'get_array_of_samples'

In [9]:

from Tokenize_Kor import isVowel
def levenshtein_distance(s1, s2):
    m, n = len(s1), len(s2)
    # DP 테이블 초기화
    dp = [[0] * (n+1) for _ in range(m+1)]
    # DP 테이블 첫 번째 행과 열 초기화
    for i in range(1, m+1):
        dp[i][0] = i
    for j in range(1, n+1):
        dp[0][j] = j
    # DP 테이블 채우기
    for i in range(1, m+1):
        for j in range(1, n+1):
            if s1[i-1] == s2[j-1]:
                dp[i][j] = dp[i-1][j-1]
            else:
                penalty = 1
                s1_vowel = isVowel(s1[i-1])
                s2_vowel = isVowel(s2[j-1])
                if(s1_vowel == s2_vowel):
                    penalty = 0.5

                dp[i][j] = min(dp[i-1][j], dp[i][j-1], dp[i-1][j-1]) + penalty
    return dp[m][n], dp


origintext = 'ㅊㅣㄴㄱㅜㄹㅏㅇ'
predtext ='ㅊㅣㄴㄱㅜㄹㅎㅏㅇ'
print(origintext)
print(predtext)

ㅊㅣㄴㄱㅜㄹㅏㅇ
ㅊㅣㄴㄱㅜㄹㅎㅏㅇ


레벤슈타인 거리를 나타내는 matric을 구한다.<br>
그리고 열과 행에서 유일하게 최솟값을 가질 때, 해당 matric에서 페널티가 적은 경로로 음소를 매칭한다.<br>
멅티스레드 가능성...?<br>

In [10]:
distance, dp = levenshtein_distance(origintext, predtext)
print(f"레벤슈타인 거리: {distance}")
np.set_printoptions(threshold=np.inf)
dp_array = np.array(dp)
print("DP 매트릭스:")
print(dp_array)

레벤슈타인 거리: 0.5
DP 매트릭스:
[[0.  1.  2.  3.  4.  5.  6.  7.  8.  9. ]
 [1.  0.  1.  1.5 2.  3.  3.5 4.  5.  5.5]
 [2.  1.  0.  1.  2.  2.5 3.5 4.5 4.5 5.5]
 [3.  1.5 1.  0.  0.5 1.5 2.  2.5 3.5 4. ]
 [4.  2.  2.  0.5 0.  1.  1.5 2.  3.  3.5]
 [5.  3.  2.5 1.5 1.  0.  1.  2.  2.5 3.5]
 [6.  3.5 3.5 2.  1.5 1.  0.  0.5 1.5 2. ]
 [7.  4.5 4.  3.  2.5 1.5 1.  1.  0.5 1.5]
 [8.  5.  5.  3.5 3.  2.5 1.5 1.5 1.5 0.5]]
